### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/102_TROCAS7_Fusion_Apr2021_DENOVO_161/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/102_TROCAS7_Fusion_Apr2021_DENOVO_161


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks102 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/102_TROCAS7_Fusion_Apr2021_DENOVO_161/102_TROCAS7_Fusion_Apr2021_DENOVO_161_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks102))

print(peaks102.columns)

# These columns mess things up- get rid of them

del peaks102['Fraction']
del peaks102['Scan']
del peaks102['Source File']
del peaks102['Tag Length']
del peaks102['PTM']
del peaks102['tag (>=0%)']
del peaks102['mode']
del peaks102['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks102.columns = columns

mean_len = peaks102['length'].mean()
print(mean_len)

# look at the dataframe
peaks102.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 235
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
10.540425531914893


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,VVEVSLPR,97,8,449.7717,2,66.99,1400000.0,897.5283,0.5
1,LSSPATLNSR,97,10,523.2870,2,45.14,69400000.0,1044.5564,3.0
2,LSSPATLDSR,97,10,523.7781,2,48.92,517000.0,1045.5403,1.3
3,LSSPATLNSR,97,10,523.2872,2,45.64,69400000.0,1044.5564,3.2
4,LSSPATLNSR,96,10,523.2864,2,46.14,69400000.0,1044.5564,1.8


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks102['A'] = peaks102['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks102['C'] = peaks102['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks102['D'] = peaks102['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks102['E'] = peaks102['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks102['F'] = peaks102['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks102['G'] = peaks102['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks102['H'] = peaks102['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks102 output, there will be no isoleucines (they're lumped in with leucines)
peaks102['I'] = peaks102['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks102['K'] = peaks102['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks102['L'] = peaks102['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks102['M'] = peaks102['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks102['N'] = peaks102['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks102['P'] = peaks102['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks102['Q'] = peaks102['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks102['R'] = peaks102['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks102['S'] = peaks102['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks102['T'] = peaks102['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks102['V'] = peaks102['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks102['W'] = peaks102['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks102['Y'] = peaks102['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks102['c-carb'] = peaks102['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks102['m-oxid'] = peaks102['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks102['n-deam'] = peaks102['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks102['q-deam'] = peaks102['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks102['stripped_peptide'] = peaks102['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks102['stripped_length'] = peaks102['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks102['ptm-total'] = peaks102['c-carb'] + peaks102['m-oxid'] + peaks102['n-deam'] + peaks102['q-deam']

# calculate NAAF numerator for each peptide k
peaks102['NAAF_num.'] = peaks102['Area'] / peaks102['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50.csv")

# check out the results
peaks102.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,VVEVSLPR,97,8,449.7717,2,66.99,1400000.0,897.5283,0.5,0,...,0,0,0,0,0,0,VVEVSLPR,8,0,175000.0
1,LSSPATLNSR,97,10,523.2870,2,45.14,69400000.0,1044.5564,3.0,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,6940000.0
2,LSSPATLDSR,97,10,523.7781,2,48.92,517000.0,1045.5403,1.3,1,...,0,0,0,0,0,0,LSSPATLDSR,10,0,51700.0
3,LSSPATLNSR,97,10,523.2872,2,45.64,69400000.0,1044.5564,3.2,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,6940000.0
4,LSSPATLNSR,96,10,523.2864,2,46.14,69400000.0,1044.5564,1.8,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,6940000.0


In [5]:
# keep only stripped peptide I/L and NAAF
dn_102 = peaks102[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_102.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/102B_BY_T00_GF_DN50_ILnaafs.csv")

dn_102.head()

,stripped_peptide,Area,NAAF_num.
0,VVEVSLPR,1400000.0,175000.0
1,LSSPATLNSR,69400000.0,6940000.0
2,LSSPATLDSR,517000.0,51700.0
3,LSSPATLNSR,69400000.0,6940000.0
4,LSSPATLNSR,69400000.0,6940000.0


In [6]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks102['A'].sum(),
        'C': peaks102['C'].sum(),
        'D': peaks102['D'].sum(),
        'E': peaks102['E'].sum(),
        'F': peaks102['F'].sum(),
        'G': peaks102['G'].sum(),
        'H': peaks102['H'].sum(),
        'I': peaks102['I'].sum(),
        'K': peaks102['K'].sum(),
        'L': peaks102['L'].sum(),
        'M': peaks102['M'].sum(),
        'N': peaks102['N'].sum(),
        'P': peaks102['P'].sum(),
        'Q': peaks102['Q'].sum(),
        'R': peaks102['R'].sum(),
        'S': peaks102['S'].sum(),
        'T': peaks102['T'].sum(),
        'V': peaks102['V'].sum(),
        'W': peaks102['W'].sum(),
        'Y': peaks102['Y'].sum(),
        'c-carb': peaks102['c-carb'].sum(),
        'm-oxid': peaks102['m-oxid'].sum(),
        'n-deam': peaks102['n-deam'].sum(),
        'q-deam': peaks102['q-deam'].sum(),
        'Total area': peaks102['Area'].sum(),
        'Total length': peaks102['stripped_length'].sum()
       }

totalpeaks102 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks102['% C w/ carb'] = totalpeaks102['c-carb'] / totalpeaks102['C'] 

# calculate percentage of M's that are oxidized
totalpeaks102['% M w/ oxid'] = totalpeaks102['m-oxid'] / totalpeaks102['M'] 

# calculate percentage of N's that are deamidated
totalpeaks102['% N w/ deam'] = totalpeaks102['n-deam'] / totalpeaks102['N'] 

# calculate percentage of N's that are deamidated
totalpeaks102['% Q w/ deam'] = totalpeaks102['q-deam'] / totalpeaks102['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks102['NAAF denom.'] = totalpeaks102['Total area'] / totalpeaks102['Total length']

# write modified dataframe to new txt file
totalpeaks102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50_totals.csv")

totalpeaks102.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,147,56,34,74,86,112,181,0,154,291,...,32,12,1,1.978899e+09,2354,1.0,0.210526,0.126316,0.04,840653.784257


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 840653.784257

# use NAAF >50% ALC to get NAAF factor
peaks102['NAAF factor'] = (peaks102['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_102 = peaks102[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_102['A-NAAF50'] = peaksAAPTM_102['A'] * peaks102['NAAF factor']
peaksAAPTM_102['C-NAAF50'] = peaksAAPTM_102['C'] * peaks102['NAAF factor']
peaksAAPTM_102['D-NAAF50'] = peaksAAPTM_102['D'] * peaks102['NAAF factor']
peaksAAPTM_102['E-NAAF50'] = peaksAAPTM_102['E'] * peaks102['NAAF factor']
peaksAAPTM_102['F-NAAF50'] = peaksAAPTM_102['F'] * peaks102['NAAF factor']
peaksAAPTM_102['G-NAAF50'] = peaksAAPTM_102['G'] * peaks102['NAAF factor']
peaksAAPTM_102['H-NAAF50'] = peaksAAPTM_102['H'] * peaks102['NAAF factor']
peaksAAPTM_102['I-NAAF50'] = peaksAAPTM_102['I'] * peaks102['NAAF factor']
peaksAAPTM_102['K-NAAF50'] = peaksAAPTM_102['K'] * peaks102['NAAF factor']
peaksAAPTM_102['L-NAAF50'] = peaksAAPTM_102['L'] * peaks102['NAAF factor']
peaksAAPTM_102['M-NAAF50'] = peaksAAPTM_102['M'] * peaks102['NAAF factor']
peaksAAPTM_102['N-NAAF50'] = peaksAAPTM_102['N'] * peaks102['NAAF factor']
peaksAAPTM_102['P-NAAF50'] = peaksAAPTM_102['P'] * peaks102['NAAF factor']
peaksAAPTM_102['Q-NAAF50'] = peaksAAPTM_102['Q'] * peaks102['NAAF factor']
peaksAAPTM_102['R-NAAF50'] = peaksAAPTM_102['R'] * peaks102['NAAF factor']
peaksAAPTM_102['S-NAAF50'] = peaksAAPTM_102['S'] * peaks102['NAAF factor']
peaksAAPTM_102['T-NAAF50'] = peaksAAPTM_102['T'] * peaks102['NAAF factor']
peaksAAPTM_102['V-NAAF50'] = peaksAAPTM_102['V'] * peaks102['NAAF factor']
peaksAAPTM_102['W-NAAF50'] = peaksAAPTM_102['W'] * peaks102['NAAF factor']
peaksAAPTM_102['Y-NAAF50'] = peaksAAPTM_102['Y'] * peaks102['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_102['ccarb-NAAF50'] = peaksAAPTM_102['c-carb'] * peaksAAPTM_102['NAAF factor']
peaksAAPTM_102['moxid-NAAF50'] = peaksAAPTM_102['m-oxid'] * peaksAAPTM_102['NAAF factor']
peaksAAPTM_102['ndeam-NAAF50'] = peaksAAPTM_102['n-deam'] * peaksAAPTM_102['NAAF factor']
peaksAAPTM_102['qdeam-NAAF50'] = peaksAAPTM_102['q-deam'] * peaksAAPTM_102['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50_naaf.csv")

peaksAAPTM_102.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,VVEVSLPR,0.208171,0,0,0,1,0,0,0,0,...,0.208171,0.208171,0.000000,0.624514,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,8.255479,1,0,0,0,0,0,0,0,...,8.255479,24.766438,8.255479,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLDSR,0.061500,1,0,1,0,0,0,0,0,...,0.061500,0.184499,0.061500,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,LSSPATLNSR,8.255479,1,0,0,0,0,0,0,0,...,8.255479,24.766438,8.255479,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,LSSPATLNSR,8.255479,1,0,0,0,0,0,0,0,...,8.255479,24.766438,8.255479,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_102['NAAF factor'].sum(),
        'A': peaksAAPTM_102['A-NAAF50'].sum(),
        'C': peaksAAPTM_102['C-NAAF50'].sum(),
        'D': peaksAAPTM_102['D-NAAF50'].sum(),
        'E': peaksAAPTM_102['E-NAAF50'].sum(),
        'F': peaksAAPTM_102['F-NAAF50'].sum(),
        'G': peaksAAPTM_102['G-NAAF50'].sum(),
        'H': peaksAAPTM_102['H-NAAF50'].sum(),
        'I': peaksAAPTM_102['I-NAAF50'].sum(),
        'K': peaksAAPTM_102['K-NAAF50'].sum(),
        'L': peaksAAPTM_102['L-NAAF50'].sum(),
        'M': peaksAAPTM_102['M-NAAF50'].sum(),
        'N': peaksAAPTM_102['N-NAAF50'].sum(),
        'P': peaksAAPTM_102['P-NAAF50'].sum(),
        'Q': peaksAAPTM_102['Q-NAAF50'].sum(),
        'R': peaksAAPTM_102['R-NAAF50'].sum(),
        'S': peaksAAPTM_102['S-NAAF50'].sum(),
        'T': peaksAAPTM_102['T-NAAF50'].sum(),
        'V': peaksAAPTM_102['V-NAAF50'].sum(),
        'W': peaksAAPTM_102['W-NAAF50'].sum(),
        'Y': peaksAAPTM_102['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_102['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_102['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_102['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_102['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 840653.784257

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,279.400884,185.568186,12.674206,4.322986,18.324224,18.372602,34.738623,34.415693,0.0,17.551484,...,17.625501,12.674206,6.133766,5.383744,0.104086,1.0,0.255427,0.108784,0.022123,0.000332


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks102[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/102_BY_T00_GF_DN/102B_BY_T00_GF_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks102['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks102[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 235
# nonredundant stripped Peaks peptides >50% ALC 145
average peptide length Peaks peptides >50% ALC 10.017021276595745
# redundant Peaks peptides >50% ALC 235
# nonredundant Peaks peptides 145


,stripped_peptide
0,VVEVSLPR
1,LSSPATLNSR
2,LSSPATLDSR
3,LSSPATLNSR
4,LSSPATLNSR
